In [2]:
def canny_edge_detector(image):
    edged = cv2.Canny(image, 50, 150)
    return edged

def get_roi(image):
    height = image.shape[0]
    width = image.shape[1]
    # Defining Triangular ROI: The values will change as per your camera mounts
    triangle = np.array([[(200, height), (1100, height), (550, 250)]])
    # creating black image same as that of input image
    black_image = np.zeros_like(image)
    # Put the Triangular shape on top of our Black image to create a mask
    mask = cv2.fillPoly(black_image, triangle, 255)
    # applying mask on the original image
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def get_lines(image):
    # lines=cv2.HoughLinesP(image, bin_size, precision, threshold, dummy 2d array--no use, minLineLength, maxLineGap)
    # let's take bin size to be 2 pixels
    # let's take precision to be 1 degree = pi/180 radians
    # threshold is the votes that a bin should have to be accepted to draw a line
    # minLineLength --the minimum length in pixels a line should have to be accepted.
    # maxLineGap --the max gap between 2 broken lines which we allow for 2 lines to be connected together.
    lines = cv2.HoughLinesP(image, 2, np.pi / 180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    return lines

# display lines over an image
def display_lines(image, lines):
    if lines is not None:
        for line in lines:
            # print(line) --output like [[704 418 927 641]] this is a 2d array representing [[x1, y1, x2, y2]] for each line
            x1, y1, x2, y2 = line.reshape(4)  # converting to a 1d array []

            # draw a line over the black image --(255, 0, 0) tells we want to draw a blue line (b, g, r) values 10 is the line thickness
            cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return image

def get_line_coordinates_from_parameters(image, line_parameters):
    slope = line_parameters[0]
    intercept = line_parameters[1]
    y1 = image.shape[0]  # since the line will always start from the bottom of the image
    y2 = int(y1 * (3.4 / 5))  # some random point at 3/5
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1, y1, x2, y2])

# Averages all the left and right lines found for a lane and returns a single left and right line for the lane
def get_smooth_lines(image, lines):
    left_fit = []  # will hold m, c parameters for left side lines
    right_fit = []  # will hold m, c parameters for right side lines

    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        # polyfit gives slope(m) and intercept(c) values from input points
        # the last parameter 1 is for linear..so it will give linear parameters m, c
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]

        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))

    # take averages of all intercepts and slopes separately and get 1 single value for slope, intercept
    # axis=0 means vertically...see it's always (row, column)...so the row is always at the 0 position.
    # so axis 0 means over row(vertically)
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)

    # now we have got m, c parameters for the left and right line, we need to know x1, y1 x2, y2 parameters
    left_line = get_line_coordinates_from_parameters(image, left_fit_average)
    right_line = get_line_coordinates_from_parameters(image, right_fit_average)
    return np.array([left_line, right_line])


In [3]:
video_feed = cv2.VideoCapture("test_video.mp4")

In [ ]:
from google.colab import files

import numpy as np
import io
import base64
from IPython.display import HTML, display
from google.colab.patches import cv2_imshow

# ... (Rest of the code remains unchanged) ...

video_feed = cv2.VideoCapture("test_video.mp4")

# Get the video properties
fps = int(video_feed.get(cv2.CAP_PROP_FPS))
frame_width = int(video_feed.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_feed.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_video_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

try:
    while video_feed.isOpened():
        (status, image) = video_feed.read()

        if not status:
            break

        edged_image = canny_edge_detector(image)  # Step 1
        roi_image = get_roi(edged_image)  # Step 2

        lines = get_lines(roi_image)  # Step 3

        smooth_lines = get_smooth_lines(image, lines)  # Step 5
        image_with_smooth_lines = display_lines(image, smooth_lines)  # Step 4

        # Write the frame to the output video
        output_video.write(image_with_smooth_lines)

        cv2_imshow(image_with_smooth_lines)

        if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to stop the video
            break

    video_feed.release()
    output_video.release()
    cv2.destroyAllWindows()

    # Display the output video in Colab
    video = io.open(output_video_path, 'r+b').read()
    display(HTML(f"""
    <video width="{frame_width}" height="{frame_height}" controls>
        <source src="data:video/mp4;base64,{base64.b64encode(video).decode()}" type="video/mp4">
    </video>
    """))

    files.download(output_video_path)

except Exception as e:
    print("An error occurred:", e)